In [79]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

In [80]:
from ipyleaflet import Map
basic_map = Map(zoom = 1)
basic_map

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [81]:
import gpxpy
#gpx_file_path = 'Gran_Fondo_Porto_SantElpidio_2022.gpx'
gpx_file_path = 'Cesena_Fontanella_Pantani_Sorrivoli_Centenara_Cesena.gpx'

with open(gpx_file_path) as fh:
    gpxFile = gpxpy.parse(fh)

In [82]:
from statistics import mean
#import ipyleaflet
from ipyleaflet import basemaps, FullScreenControl, LayerGroup, \
    MeasureControl, Polyline, Marker, CircleMarker, WidgetControl
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, \
    Label, Output, IntSlider, Layout, Image, link
from bqplot import pyplot as plt
from bqplot import *

In [83]:
#def plot_map(gpx):
"""
Plot the GPS trace on a map
"""
from ipyleaflet import Map

points = [p.point for p in gpxFile.get_points_data(distance_2d=True)]
mean_lat = mean(p.latitude for p in points)
mean_lng = mean(p.longitude for p in points)

# create the map
m = Map(center=(mean_lat, mean_lng), zoom=12, basemap=basemaps.Stamen.Terrain)

# show trace
line = Polyline(locations=[[[p.latitude, p.longitude] for p in points],],
                color = "red", fill=False)
m.add_layer(line)

# add markers
waypoints = [
    Marker(location=(point.latitude, point.longitude), title=point.name,
           popup=HTML(value=point.name), draggable=False)
    for point in gpxFile.waypoints
]
waypoints_layer = LayerGroup(layers=waypoints)
#m.add_layer(waypoints_layer)

# add a checkbox to show / hide waypoints
waypoints_checkbox = Checkbox(value=True, description='Show Waypoints')

def update_visible(change):
    for p in waypoints:
        p.visible = change['new']

waypoints_checkbox.observe(update_visible, 'value')
waypoint_control = WidgetControl(widget=waypoints_checkbox, position='bottomright')
m.add_control(waypoint_control)

# enable full screen mode
m.add_control(FullScreenControl())

# add measure control
measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)
m.add_control(measure)

#return m
m

Map(center=[44.079069151812824, 12.218333058224683], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [84]:
# https://www.youtube.com/watch?v=jAKUPm-Cul0
#j = 90
#gpxFile.get_points_data(distance_2d=True)[j]#.tracks().segments[j]

In [85]:
#def plot_elevation(gpx):
"""
Return an elevation graph for the given gpx trace
"""
points = gpxFile.get_points_data(distance_2d=True)
px = [p.distance_from_start / 1000 for p in points]
py = [p.point.elevation for p in points]

x_scale, y_scale = LinearScale(), LinearScale()
x_scale.allow_padding = False
x_ax = Axis(label='Distance (km)', scale=x_scale)
y_ax = Axis(label='Elevation (m)', scale=y_scale, orientation='vertical')

lines = Lines(x=px, y=py, scales={'x': x_scale, 'y': y_scale})

u = 90
elevation = Figure(title='Elevation Chart %d' %u, axes=[x_ax, y_ax], marks=[lines])


elevation.layout.width = 'auto'
elevation.layout.height = 'auto'
elevation.layout.min_height = '300px'

#elevation.interaction = IndexSelector(scale=x_scale)
#return elevation
elevation

Figure(axes=[Axis(label='Distance (km)', scale=LinearScale(allow_padding=False)), Axis(label='Elevation (m)', …

In [86]:
import pandas as pd

j = 0
points = []
for segment in gpxFile.tracks[j].segments:
    for p in segment.points:
        points.append({
            'time': p.time,
            'latitude': p.latitude,
            'longitude': p.longitude,
            'elevation': p.elevation,
        })
coords = pd.DataFrame.from_records(points)

from datetime import datetime

timeList = [ datetime.timestamp( u ) for u in coords['time'] ]
DeltaTime = np.diff(timeList)
DeltaTime = np.append([0], DeltaTime)
coords['DeltaTime'] = pd.Series( DeltaTime )

space = ( (coords['latitude']*111.139)**2 + (coords['longitude']*111.139 )**2 + \
 coords['elevation']**2 )**.5

DeltaSpace = np.append([0], np.diff(space) )

coords['DeltaSpace'] = pd.Series( np.diff(space) )
coords['speed'] = 3.6*coords['DeltaSpace']/coords['DeltaTime']
coords

,time,latitude,longitude,elevation,DeltaTime,DeltaSpace,speed
0,2022-10-02 15:13:49+00:00,44.140379,12.238573,34.0,0.0,-0.001283,-inf
1,2022-10-02 15:13:50+00:00,44.140386,12.238527,33.9,1.0,-0.001238,-0.004456
2,2022-10-02 15:13:52+00:00,44.140402,12.238450,33.8,2.0,-0.000494,-0.000890
3,2022-10-02 15:13:53+00:00,44.140413,12.238416,33.7,1.0,-0.000473,-0.001705
4,2022-10-02 15:13:55+00:00,44.140430,12.238361,33.6,2.0,-0.001202,-0.002164
...,...,...,...,...,...,...,...
5594,2022-10-02 16:53:51+00:00,44.139665,12.238379,34.1,1.0,0.007173,0.025824
5595,2022-10-02 16:53:52+00:00,44.139722,12.238415,34.1,1.0,0.006121,0.022034
5596,2022-10-02 16:53:53+00:00,44.139770,12.238448,34.1,1.0,-0.000464,-0.001672
5597,2022-10-02 16:53:55+00:00,44.139764,12.238454,34.1,2.0,0.001808,0.003254
